# Question Answering Models (QA)

## A Deep Learning Model that can answer all your questions.... if good of course.

### There are 2 different categories of QA Modeling
- Domain - systems that are constrained by the input data; we have open and closed systems
    - <b> Open domain systems </b> are for broad questions, not specific to what category of discussion (Wikipedia, World wide web, Alexa, etc...)
    - <b> Closed domain systems </b> are more narrow in its vocabulary and focus on a specific industry or topic (Football, Finance, Tech, Law etc..) 
- Question Type - Open ended questions, Yes/No questions, inference questions etc...)
    - Once you determine what type of system you want to establish, you then need to figure out which question type you want your model to focus on



### Types of Question Answering Models
- <i> Extractive Question Answering </i> is a deep learning model that can provide an answer when given a corpus of text (i.e context). So, when you provide a question to the model, the model then "searches" the documents to pinpoint the best answer to the question. It's essentially a searching tool in many ways... 
- <i> Open Generative Question Answering </i> is a deep learning model that generates text based on context. So, unlike the extractive question answering model, the answer does not <b> literally </b> have to be in the text.
- <i> Closed Generative Question Answering </i> is a deep learning model where no context is provided and the answer is generated by the model. 

More information about the 3 subsets of Question Answering Modelings can be found [at HuggingFace.co](https://huggingface.co/tasks/question-answering) -- we'll be <b> focusing more on the <i> Extractive Question Answering </i> and fine-tuning aspects in this notebook. </b>

- The QA datasets are laboriously tailored and tagged to fit the "mold" of a QA model. A training dataset will look something like this: 
    - _Also note that QA training data does not necessarily need an associated answer._

### Resources
- NLP-Progress of [current state of the art NLP tasks](http://nlpprogress.com/english/question_answering.html) --> Contains QA datasets commonly used for state of the art QA NLP models. 

- [HuggingFace repositories](https://huggingface.co/); This houses many popular NLP QA Models, some of which I will use in this notebook.

- [QA metadata format explanation](https://simpletransformers.ai/docs/qa-data-formats/)

# Extractive QA Model Structure and Use

[Check out the BERT Video I did here!](https://www.youtube.com/watch?v=72Ylk77PqR8)

- The QA Models are essentially extensions of the BERT model with slightly different ouput layers
- Interested in the base mode [BERT?](https://www.geeksforgeeks.org/explanation-of-bert-model-nlp/); Here is the actual [paper of RoBERTa: A Robustly Optimized BERT Pretraining Approach](https://arxiv.org/abs/1907.11692)
- Link to [documentation for RobERTa](https://www.geeksforgeeks.org/overview-of-roberta-model/)
    - RoBERTa is a new and improved version of BERT
        - Removes the Next Sentence Prediction (NSP) objective
        - Trained on bigger batch sizes & longer sequences
        - Dynamically changes the masking pattern
        - TRAINED on a large corpus of English data with <b> no </b> labeling whatsoever. (just the raw texts)
            - Masks 15% of the input; RoBERTa runs the entire masked sentence through the model and the model attempts to predict the masked terms correctly.
            - This is where the QA model learns the context and have a basic understanding to the language modeling!

Using the very popular question-answering model: **RoBERTa-base for QA** 

    "This is the roberta-base model, fine-tuned using the SQuAD2.0 dataset. It's been trained on question-answer pairs, including unanswerable questions, for the task of Question Answering." - deepset.ai

In [1]:
"""https://huggingface.co/deepset/roberta-base-squad2"""
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# QA Metrics Material
- [Metrics on QA (implementation and explaination)](https://qa.fastforwardlabs.com/methods/background/2020/04/28/Intro-to-QA.html)

In [2]:
res # score == F1 Score: it's computed on the individual words in the prediction vs the true words provided in context

{'score': 0.2117140293121338,
 'start': 59,
 'end': 84,
 'answer': 'gives freedom to the user'}

# RoBERTa Architecture
if you are curious...

In [5]:
model

RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm):

# How to fine-tune a QA Model?
- Definitley need a GPU. Else, you are looking at a fine-tuning phase that is at least 10 times slower to train.
- Let's leverage an already built training script.
    - [Located here](https://github.com/huggingface/transformers/blob/b90745c5901809faef3136ed09a689e7d733526c/examples/run_squad.py); The execution is in the cell below.
    
What is this script doing?
- Gets whichever pretrained model you want to use (we are using RoBERTa in this case, but you can use a different pretrained model)
- Input dataset is converted into features
    - The featured dataset is saved in cache, so you don't have to necessarily rerun this process once more for this model.
- Ensure that the <b> --do_train </b> is enabled; This commences the training.
- When training is done, the outputs of the model are saved in a <b> output_dir / checkpoint - step_number</b>

In [ ]:
"""You could go through this route and run your training script here via command line with the associated parameters
and the script."""
## More information on how to do this can be found here: https://qa.fastforwardlabs.com/pytorch/hugging%20face/wikipedia/bert/transformers/2020/05/19/Getting_Started_with_QA.html
# Download training script from the transformers library
# !curl -L -O https://github.com/huggingface/transformers/blob/b90745c5901809faef3136ed09a689e7d733526c/examples/run_squad.py



### BUT, but there are some functions that enable us to do this without going the command line route.

# Using Popular Libraries

In [5]:
from datasets import load_dataset
# More documentation about the dataset can be found here: https://huggingface.co/datasets/viewer/?dataset=squad
# This is essentially a wrapper for the segmented data.
# The SQuAD dataset is a popular dataset based on wikipedia articles where there is an answer in the context provided.
# (different from SQuAD2.0)
squad = load_dataset("squad") 

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to C:\Users\Spenc\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [7]:
# More specific information about the dataset can be found here: https://huggingface.co/datasets/squad#data-instances
squad["train"][0]
# id -> hash of the context
# title -> Document where the context resides
# Context -> Information where the answer resides
# Question -> What question are you trying to find the answer to?
# Answers -> What is the answer to the question? And the location on where in the text the answer begins (span)

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [8]:
# Preprocess the data to a BERT format
def preprocess_function(examples):
    """Courtesy of https://huggingface.co/docs/transformers/tasks/question_answering"""
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [9]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Parameter 'function'=<function preprocess_function at 0x0000023380587790> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [10]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

# CPU Training Result... Let's think about GPU's okay?
<img src="CPU_Training.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" />

In [11]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_name) # remember that model_name is deepset/roberta-base-squad2

In [12]:
# Let's start training!
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

c:\users\spenc\anaconda3\envs\nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 87599
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 16425


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

# How to train a QA model from scratch?
- Well, its a matter of plugging in a large corpus of text to an unweighted BERT Model.
    - This is the pre-training phase of the BERT model.
    - Check out this [link on how to pretrain your very own model!](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/zh/latest/nlp/bert_pretraining.html)